# Modelling, Analysis and Benchmarking of a Simple Unbalanced LV Network (with Neutral)

## Introduction

This tutorial will demonstrate how to model an unbalanced LV network with a neutral wire in 
_Roseau Load Flow (RLF)_ solver. We will replicate the network which was initially specified in _OpenDSS_ syntax in 
_RLF_ and compare the results obtained to benchmark their capabilities.

Before attempting this tutorial, you should have finished "Tutorial 1" in this repository for a basic knowledge of 
how the _RLF_ solver works. The network consists of an MV bus, a MV/LV, $\Delta$-Y transformer (11kV/0.4kV, 250 kVA) 
between the source bus and bus A, a 240 mm² 3-phase line connecting buses A and B, and three 16 mm² single-phase 
lines connecting bus B with buses C, D and E each of which serves as a connection point for a house.

<center> <img style="float: middle;" 
          src="../images/LV_Network_With_Neutral.png" 
          alt="Simple LV network"
          width="40%"> 
</center>

**<center> Figure 1. Simple LV Network with a Neutral wire </center>**

The details for the loads in the network are given in the table below.
| Load Name | Phases | Connected bus | Peak Demand (kW) | PF |
| :-------- | :----- | :------------ | :--------------- | :--- |
| Load_1 | 1 | C | 7 | 0.95 |
| Load_2 | 1 | D | 6 | 0.95 |
| Load_3 | 1 | E | 8 | 0.95 |

As this tutorial is very similar to the previous one (using a earth return system), we just define the networks and 
compare the results without detailed explanations.

In [ ]:
import dss
import numpy as np
import roseau.load_flow as rlf
from rich.table import Table
from rich.text import Text

## Open DSS


In [ ]:
# Set up dss_engine
dss_engine = dss.DSS
DSSText = dss_engine.Text
DSSCircuit = dss_engine.ActiveCircuit
DSSSolution = dss_engine.ActiveCircuit.Solution
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue
dss_engine.AllowForms = 0

# Network Modelling - Creating a Circuit
DSSText.Command = "Clear"
DSSText.Command = "Set DefaultBaseFrequency=50"
DSSText.Command = "New Circuit.Simple_LV_Network"
DSSText.Command = "Edit vsource.source bus1=sourceBus basekv=11 pu=1.0 phases=3"

# Adding the 11kV/0.4kV Transformer
DSSText.Command = "New transformer.LVTR Buses=[sourceBus, A.1.2.3.4] Conns=[delta wye] KVs=[11, 0.4] KVAs=[250 250] %Rs=0.00 xhl=2.5 %loadloss=0 "
# DSSText.Command = "dump transformer.LVTR debug"

# Creating the linecodes
DSSText.Command = "new linecode.240sq nphases=4 R1=0.127 X1=0.072 R0=0.342 X0=0.089 units=km"
DSSText.Command = "new linecode.16sq nphases=2 R1=1.15 X1=0.083 R0=1.2 X0=0.083 units=km"

# Creating the 400V and 230V lines
DSSText.Command = "new line.A_B bus1=A.1.2.3.4 bus2=B.1.2.3.4 length=1 phases=4 units=km linecode=240sq"
DSSText.Command = "new line.B_L1 bus1=B.1.4 bus2=C.1.2 length=0.01 phases=2 units=km linecode=16sq"
DSSText.Command = "new line.B_L2 bus1=B.2.4 bus2=D.1.2 length=0.01 phases=2 units=km linecode=16sq"
DSSText.Command = "new line.B_L3 bus1=B.3.4 bus2=E.1.2 length=0.01 phases=2 units=km linecode=16sq"
# DSSText.Command = "dump linecode.240sq debug"
# DSSText.Command = "dump line.A_B debug"

# Connecting loads to a bus
# Note: I (Ali) couldn't make it work with phases=2 for loads. It will need more digging
# into the OpenDSS model to understand the discrepancy between our phase-neutral load
# definition and theirs. For the time being, keeping phases=1 works fine as long as
# the correct bus phases are used in the load definition
DSSText.Command = "new load.Load_1 bus1=C.1.2 phases=1 kV=(0.4 3 sqrt /) kW=7 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_2 bus1=D.1.2 phases=1 kV=(0.4 3 sqrt /) kW=6 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_3 bus1=E.1.2 phases=1 kV=(0.4 3 sqrt /) kW=8 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"

# Set the Control mode and the Voltage bases
DSSText.Command = "set controlmode=static"
DSSText.Command = "set mode=snapshot"
DSSText.Command = "Set VoltageBases=[11 0.4]"
DSSText.Command = "calcvoltagebases"

In [ ]:
# Run the power flow simulation
DSSSolution.Solve()
if DSSSolution.Converged:
    print("The Circuit was Successfully Solved")
else:
    raise RuntimeError("DID NOT CONVERGE")

In [ ]:
# Extract the results
table_titles = []
table_opendss_results = []
active_power_demand_title = "Active power demand (P)"
reactive_power_demand_title = "Reactive power demand (Q)"
for active_load in ("Load_1", "Load_2", "Load_3"):
    table_titles.append(f"Power - {active_load}")
    table_opendss_results.append(None)

    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    print(f"{active_load}:  ")
    pp, pn = DSSCircuit.ActiveElement.Powers[0::2]
    active_power_demand = f"{pp+pn:.3f} kW"
    print(f"{active_power_demand_title} = {active_power_demand} ")
    table_titles.append(active_power_demand_title)
    table_opendss_results.append(active_power_demand)

    qp, qn = DSSCircuit.ActiveElement.Powers[1::2]
    reactive_power_demand = f"{qp+qn:.3f} kVAr"
    print(f"{reactive_power_demand_title} = {reactive_power_demand}")
    table_titles.append(reactive_power_demand_title)
    table_opendss_results.append(reactive_power_demand)
print()


for active_load in ("Load_1", "Load_2", "Load_3"):
    table_titles.append(f"Voltage - {active_load}")
    table_opendss_results.append(None)

    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    bus_name = DSSCircuit.ActiveElement.Properties("bus1").Val
    DSSCircuit.SetActiveBus(bus_name)
    voltages = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
    (v1,) = np.abs(voltages[:-1] - voltages[-1])
    voltage_title = f"The voltage of the bus connected to {active_load}"
    voltage_value = f"{v1:.3f} pu"
    print(f"{voltage_title} = {voltage_value}")
    table_titles.append(voltage_title)
    table_opendss_results.append(voltage_value)
print()

active_bus = "A"
DSSCircuit.SetActiveBus(active_bus)
print(f"Voltage magnitudes at bus {active_bus}:  ")
table_titles.append(f"Bus {active_bus}")
table_opendss_results.append(None)
voltages = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
v1, v2, v3 = np.abs(voltages[:-1] - voltages[-1])
for i, v in enumerate((v1, v2, v3)):
    voltage_title = f"Voltage magnitude - phase {i + 1}"
    table_titles.append(voltage_title)
    voltage_value = f"{v:.3f} pu"
    table_opendss_results.append(voltage_value)
    print(f"{voltage_title} = {voltage_value}")
print()

DSSCircuit.SetActiveElement("transformer.LVTR")
print("Results of the transformer LVTR:")
table_titles.append("Results of the transformer LVTR:")
table_opendss_results.append(None)

# Indices [0,4[ are primary powers, indices [4, 8[ are secondary powers
transformer_p = DSSCircuit.ActiveElement.Powers[0::2]
transformer_q = DSSCircuit.ActiveElement.Powers[1::2]

for i in range(3):
    transformer_title = f"Active power (P) supplied to phase {i+1}"
    transformer_value = f"{abs(transformer_p[4+i]):.5f} kW"
    table_titles.append(transformer_title)
    table_opendss_results.append(transformer_value)
    print(f"{transformer_title} = {transformer_value}")

for i in range(3):
    transformer_title = f"Reactive power (Q) supplied to phase {i+1}"
    transformer_value = f"{abs(transformer_q[4+i]):.5f} kVAr"
    table_titles.append(transformer_title)
    table_opendss_results.append(transformer_value)
    print(f"{transformer_title} = {transformer_value}")

transformer_title = "Total active power (P) losses"
transformer_value = f"{abs(sum(transformer_p)):.5f} kW"
table_titles.append(transformer_title)
table_opendss_results.append(transformer_value)
print(f"{transformer_title} = {transformer_value}")

transformer_title = "Total reactive power (Q) losses"
transformer_value = f"{abs(sum(transformer_q)):.5f} kVAr"
table_titles.append(transformer_title)
table_opendss_results.append(transformer_value)
print()

line_loss = DSSCircuit.LineLosses
lines_p, lines_q = line_loss
print("Results of the Power Losses:")
table_titles.append("Line Losses")
table_opendss_results.append(None)

line_title = "Active power (P) losses"
line_value = f"{abs(lines_p):.3f} kW"
table_titles.append(line_title)
table_opendss_results.append(line_value)
print(f"{line_title} = {line_value}")

line_title = "Reactive power (Q) losses"
line_value = f"{abs(lines_q):.3f} kVAr"
table_titles.append(line_title)
table_opendss_results.append(line_value)
print(f"{line_title} = {line_value}")

## Roseau Load Flow


In [ ]:
# Buses
# -----
source_bus = rlf.Bus(id="source_bus", phases="abc")
bus_a = rlf.Bus(id="bus_A", phases="abcn")
bus_b = rlf.Bus(id="bus_B", phases="abcn")
bus_c = rlf.Bus(id="bus_C", phases="an")
bus_d = rlf.Bus(id="bus_D", phases="bn")
bus_e = rlf.Bus(id="bus_E", phases="cn")

# References
# ----------
pref_mv = rlf.PotentialRef(id="pref_mv", element=source_bus)
ground = rlf.Ground(id="ground")
ground.connect(bus=bus_a, phase="n")
pref_lv = rlf.PotentialRef(id="pref_lv", element=ground)

# Sources
# -------
un_mv = rlf.Q_(11, "kV")
un_lv = rlf.Q_(400, "V")
vs = rlf.VoltageSource(id="vs", bus=source_bus, voltages=un_mv)

# Transformers
# ------------
tp = rlf.TransformerParameters.from_open_dss(
    id="LVTR",
    conns=("delta", "wye"),
    kvs=[un_mv, un_lv],
    kvas=(250, 250),
    leadlag="euro",  # <- should be "ansi" (i.e. "Dyn1") but we don't have this in RLF yet
    xhl=2.5,
    rs=0,
    loadloss=0,
    noloadloss=0,
    imag=0,
    normhkva=None,
)
transformer = rlf.Transformer("LVTR", bus1=source_bus, bus2=bus_a, parameters=tp)

# Lines
# -----
lp_240 = rlf.LineParameters.from_open_dss(
    id="linecode-240sq",
    nphases=4,
    r1=rlf.Q_(0.127, "ohm/km"),
    x1=rlf.Q_(0.072, "ohm/km"),
    r0=rlf.Q_(0.342, "ohm/km"),
    x0=rlf.Q_(0.089, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)
lp_16 = rlf.LineParameters.from_open_dss(
    id="linecode-16sq",
    nphases=2,
    r1=rlf.Q_(1.150, "ohm/km"),
    x1=rlf.Q_(0.083, "ohm/km"),
    r0=rlf.Q_(1.200, "ohm/km"),
    x0=rlf.Q_(0.083, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)

line_ab = rlf.Line(
    "lineA_B", bus1=bus_a, bus2=bus_b, phases="abcn", parameters=lp_240, length=rlf.Q_(1, "km"), ground=ground
)
line_bc = rlf.Line(
    "lineB_C", bus1=bus_b, bus2=bus_c, phases="an", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)
line_bd = rlf.Line(
    "lineB_D", bus1=bus_b, bus2=bus_d, phases="bn", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)
line_be = rlf.Line(
    "lineB_E", bus1=bus_b, bus2=bus_e, phases="cn", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)


# Loads
# -----
def complex_power(p: float, pf: float) -> complex:
    phi = np.arccos(pf)
    q = p * np.tan(phi)
    return p + 1j * q


load1 = rlf.PowerLoad(id="load1", bus=bus_c, phases="an", powers=[complex_power(7e3, 0.95)])
load2 = rlf.PowerLoad(id="load2", bus=bus_d, phases="bn", powers=[complex_power(6e3, 0.95)])
load3 = rlf.PowerLoad(id="load3", bus=bus_e, phases="cn", powers=[complex_power(8e3, 0.95)])

en = rlf.ElectricalNetwork.from_element(initial_bus=source_bus)
en

In [ ]:
en.solve_load_flow()

In [ ]:
table_rlf_results: list[str | None] = []
for load in (load1, load2, load3):
    table_rlf_results.append(None)

    print(f"{load.id}:  ")
    load_powers = load.res_powers.m_as("kVA").sum()
    load_value = f"{load_powers.real:.3f} kW"
    print(f"{active_power_demand_title} = {load_value}")
    table_rlf_results.append(load_value)

    load_value = f"{load_powers.imag:.3f} kVAr"
    print(f"{reactive_power_demand_title} = {load_value}")
    table_rlf_results.append(load_value)
print()

for load in (load1, load2, load3):
    table_rlf_results.append(None)

    voltages_pu = abs(load.res_voltages.m[0]) / (un_lv.m_as("V") / np.sqrt(3))
    voltage_title = f"The voltage of the bus connected to {load.id}"
    voltage_value = f"{voltages_pu:.3f} pu"
    print(f"{voltage_title} = {voltage_value}")
    table_rlf_results.append(voltage_value)
print()

print("Voltage magnitudes at bus A:  ")
table_rlf_results.append(None)
bus_a_voltages_pu = abs(bus_a.res_voltages.m) / (un_lv.m_as("V") / np.sqrt(3))
for i in range(3):
    voltage_value = f"{bus_a_voltages_pu[i]:.3f} pu"
    table_rlf_results.append(voltage_value)
    print(f"Voltage magnitude - phase {i+1} = {voltage_value}")
print()

transformer_powers = transformer.res_powers[1].m_as("kVA")
transformer_power_losses = transformer.res_power_losses.m_as("kVA")
print("Results of the transformer LVTR: ")
table_rlf_results.append(None)
for i in range(3):
    transformer_value = f"{abs(transformer_powers[i].real):.5f} kW"
    table_rlf_results.append(transformer_value)
    print(f"Active power (P) supplied to phase {i+1} = {transformer_value}")
for i in range(3):
    transformer_value = f"{abs(transformer_powers[i].real):.5f} kVAr"
    table_rlf_results.append(transformer_value)
    print(f"Reactive power (Q) supplied to phase {i+1} = {transformer_value}")

transformer_value = f"{transformer_power_losses.real:.5f} kW"
table_rlf_results.append(transformer_value)
print(f"Total active power (P) losses = {transformer_value}")
transformer_value = f"{transformer_power_losses.imag:.5f} kVAr"
table_rlf_results.append(transformer_value)
print(f"Total reactive power (Q) losses = {transformer_value}")
print()

total_line_loss = sum(br.res_power_losses.m_as("kVA").sum() for br in en.lines.values())
print("Results of the Power Losses: ")
table_rlf_results.append(None)
line_value = f"{abs(total_line_loss.real):.3f} kW"
table_rlf_results.append(line_value)
print(f"Active power (P) losses = {line_value}")
line_value = f"{abs(total_line_loss.imag):.3f} kVAr"
table_rlf_results.append(line_value)
print(f"Reactive power (Q) losses = {line_value}")

## Comparison

Please find below a table which summarizes the results of the load flow solvers.

In [ ]:
table = Table(title="Comparison of results")

table.add_column("Title", justify="left", no_wrap=True)
table.add_column(Text("OpenDSS", style="italic"), justify="right", style="green")
table.add_column(Text("Roseau Load Flow", style="italic"), justify="right", style="green")


for title, opendss_value, rlf_value in zip(table_titles, table_opendss_results, table_rlf_results, strict=True):
    if opendss_value is None:
        table.add_section()
        table.add_row(Text(title, style="bold"))
    elif rlf_value == opendss_value:
        table.add_row(title, opendss_value, rlf_value)
    else:
        table.add_row(title, Text(opendss_value, style="red"), Text(rlf_value, style="red"))

table